In [28]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
plt.style.use ('ggplot')
import seaborn as sns
%matplotlib inline
sns.set_style(style="whitegrid")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

from bs4 import BeautifulSoup

import nltk, string, contractions

from gensim.corpora.dictionary import Dictionary
from gensim.models.nmf import Nmf
from gensim.models.coherencemodel import CoherenceModel

from operator import itemgetter

In [2]:
df = pd.read_csv('data/modeling_ready_microwave1')

In [3]:
df.head()

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount,clean_review
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-31,91,I have had this microwave for just over 3 year...
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-30,23,Man this thing cooks evenly and fast! It takes...
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-30,17,Works great! Have had it over a year and no su...
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-29,13,I ordered and received this several years ago ...
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-28,131,I have had this microwave for just over 2 year...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875 entries, 0 to 1874
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        1875 non-null   int64  
 1   review_id          1875 non-null   object 
 2   product_id         1875 non-null   object 
 3   star_rating        1875 non-null   float64
 4   helpful_votes      1875 non-null   float64
 5   total_votes        1875 non-null   float64
 6   verified_purchase  1875 non-null   object 
 7   review_headline    1875 non-null   object 
 8   review_body        1875 non-null   object 
 9   review_date        1875 non-null   object 
 10  review_wordcount   1875 non-null   int64  
 11  clean_review       1875 non-null   object 
dtypes: float64(3), int64(2), object(7)
memory usage: 175.9+ KB


In [5]:
df.review_date = pd.to_datetime(df.review_date)

In [6]:
df.describe()

,customer_id,star_rating,helpful_votes,total_votes,review_wordcount
count,1.875000e+03,1875.000000,1875.000000,1875.000000,1875.000000
mean,3.209401e+07,3.850133,3.107733,3.628267,80.606933
std,1.476223e+07,1.496151,20.062047,20.926332,97.269705
min,8.722000e+04,1.000000,0.000000,0.000000,1.000000
25%,1.748780e+07,3.000000,0.000000,0.000000,28.000000
50%,3.303278e+07,5.000000,1.000000,1.000000,51.000000
75%,4.624525e+07,5.000000,2.000000,2.000000,94.000000
max,5.309426e+07,5.000000,589.000000,599.000000,1152.000000


In [7]:
punct =[]
punct += list(string.punctuation)
punct += '’'
punct.remove("'")
def remove_punctuations(text):
    for punctuation in punct:
        text = text.replace(punctuation, ' ')
    return text

In [31]:
def nlp_prep(df):
    # lowercase everything
    # get rid of '\n' from whitespace
    # regex remove hyperlinks
    # remove punctuation
    # remove ' s ' from removing punctuation
    
    # lowercase everything
    df['token'] = df['clean_review'].apply(lambda x: x.lower())
    # expand contractions
    df['token'] = df['token'].apply(lambda x:[contractions.fix(word) for word in x.split()])
    df['token'] = [' '.join(map(str, l)) for l in df['token']]
    # get rid of '\n' from whitespace 
    df['token'] = df['token'].apply(lambda x: x.replace('\n', ' '))
    # regex remove hyperlinks
    df['token'] = df['token'].str.replace('http\S+|www.\S+', '', case=False)
    # remove punctuations
    df['token'] = df['token'].apply(remove_punctuations)
    # remove ' s ' that was created after removing punctuations
    df['token'] = df['token'].apply(lambda x: str(x).replace(" s ", " "))
    return df

In [32]:
nlp_prep(df)

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount,clean_review,token,lemmatized
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-31,91,I have had this microwave for just over 3 year...,i have had this microwave for just over 3 year...,"[i, have, had, this, microwave, for, just, ove..."
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-30,23,Man this thing cooks evenly and fast! It takes...,man this thing cooks evenly and fast it takes...,"[man, this, thing, cook, evenly, and, fast, it..."
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-30,17,Works great! Have had it over a year and no su...,works great have had it over a year and no su...,"[work, great, have, had, it, over, a, year, an..."
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-29,13,I ordered and received this several years ago ...,i ordered and received this several years ago ...,"[i, ordered, and, received, this, several, yea..."
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-28,131,I have had this microwave for just over 2 year...,i have had this microwave for just over 2 year...,"[i, have, had, this, microwave, for, just, ove..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,33705501,R34GY91QSWWUE4,B0009KMYGY,4.0,60.0,64.0,N,Just right for me...,The Panasonic NN-H765WF was exactly what I was...,2006-03-23,74,The Panasonic NN-H765WF was exactly what I was...,the panasonic nn h765wf was exactly what i was...,"[the, panasonic, nn, h765wf, wa, exactly, what..."
1871,39268146,R7N1LRCV8GU6L,B0009KMYHI,5.0,5.0,10.0,Y,good looking-good cooking,this microwave cooks food very evenly-no need ...,2006-03-19,21,this microwave cooks food very evenly-no need ...,this microwave cooks food very evenly no need ...,"[this, microwave, cook, food, very, evenly, no..."
1872,19314267,R15KWS9QSI7URC,B0009KMYDM,5.0,12.0,13.0,Y,Panasonic NN-H965WF Luxury full-size 2.2 cu.ft...,"its a great microwave , i use it every day , i...",2006-03-16,67,"its a great microwave , i use it every day , i...",its a great microwave i use it every day i...,"[it, a, great, microwave, i, use, it, every, d..."
1873,11400155,RK5HSUM1GNRH8,B0009KMYDM,5.0,4.0,9.0,N,Panasonic Microwave Oven NN-965WF,Have had it for only a short time but it has p...,2006-02-24,20,Have had it for only a short time but it has p...,have had it for only a short time but it has p...,"[have, had, it, for, only, a, short, time, but..."


In [10]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [11]:
keep_words = ['not', 'ain', 'aren', "aren't", 'couldn', "couldn't", 
              'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 
              'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 
              'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn',
              "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 
              'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 
              'wouldn', "wouldn't", 'fire', 'off']

In [43]:
review_stop_words = ['panasonic', 'really', 'husband', 'thanks', 'thank', 'ha', 
              'just', 'thing', 'did', 'nn', 'wa', 'yr', 'u', 'say', 'doe',
              'mom', 'christmas', 'gift', 'got', 'way', 'le', 'daughter',
              'e','not','love','good','bought','great microwave','micro',
              'great oven','microwave','product','work great','nice work',
              'work great use','work great love','feature work great',
              'unit work great','oven work great','easy use love','old oven',
              'old old','20 year old','unit', 'not', 'work']
for _ in text.ENGLISH_STOP_WORDS:
    if _ in keep_words:
        pass
    else:
        review_stop_words.append(_)

In [13]:
len(stop_words)

361

In [44]:
vectorizer = TfidfVectorizer(lowercase=True,
                             tokenizer = lemmatize_text,
                             stop_words= review_stop_words,
                             max_features = 5000,
                             min_df = 2,
                             max_df = .85,
                             ngram_range=(1, 2)
#                              preprocessor=' '.join
                            )

In [45]:
X = df['token']

In [46]:
X = vectorizer.fit_transform(X)

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['20', 'easy', 'feature', 'great', 'nice', 'old', 'oven', 'use', 'year'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [48]:
X.shape

(1875, 5000)

In [47]:
df.head()

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount,clean_review,token,lemmatized
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-31,91,I have had this microwave for just over 3 year...,i have had this microwave for just over 3 year...,"[i, have, had, this, microwave, for, just, ove..."
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-30,23,Man this thing cooks evenly and fast! It takes...,man this thing cooks evenly and fast it takes...,"[man, this, thing, cook, evenly, and, fast, it..."
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-30,17,Works great! Have had it over a year and no su...,works great have had it over a year and no su...,"[work, great, have, had, it, over, a, year, an..."
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-29,13,I ordered and received this several years ago ...,i ordered and received this several years ago ...,"[i, ordered, and, received, this, several, yea..."
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-28,131,I have had this microwave for just over 2 year...,i have had this microwave for just over 2 year...,"[i, have, had, this, microwave, for, just, ove..."


In [49]:
nmf = NMF(
    n_components=55,
    init='nndsvd'
).fit(X)

nmf_output = nmf.fit_transform(X)

In [20]:
def show_topics(vectorizer=vectorizer, lda_model=nmf, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [21]:
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=nmf, n_words=10)        

In [40]:
def topic_featuring(n_components=10, n_words=10):
    nmf = NMF(
        n_components=n_components,
        init='nndsvd'
    ).fit(X)

    nmf_output = nmf.fit_transform(X)

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=nmf, n_words=n_words)

    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

    Topics_theme = range(n_components)
    df_topic_keywords['topic_theme'] = Topics_theme
    df_topic_keywords.set_index('topic_theme', inplace=True)
    return df_topic_keywords.T

In [50]:
topic_featuring(55, 10)

topic_theme,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
Word 0,power,great,year,easy,door,old,oven,amazon,excellent,sensor,...,counter,used,junk,function,little,fit,new,happy,button,lasted
Word 1,level,great great,10,easy use,close,year old,great oven,day,excellent price,reheat,...,space,day,piece,right,bit,needed,new new,purchase,push,lasted year
Word 2,power level,great price,10 year,use,slam,replaced,old oven,review,excellent oven,sensor reheat,...,counter space,used like,piece junk,reheat function,little bit,cabinet,new model,happy purchase,minute,year
Word 3,power setting,great oven,3,clean,door close,replaced old,new oven,com,excellent value,sensor cook,...,kitchen,long used,money,learning,little loud,kitchen,old new,described,press,20
Word 4,setting,feature great,died,use great,shut,old sharp,recommend oven,amazon com,needing,setting,...,lot counter,used daily,machine,function great,loud,built,15,quite happy,push button,lasted 20
Word 5,lower,packing,second,easy clean,hard,replace,owned,return,pleased,reheat sensor,...,huge,daily,waste,beautifully,inside,color,like new,quite,start,20 year
Word 6,set,working great,3 year,easy operate,slam door,replace old,oven worked,shipping,plugged,hot,...,size,used day,waste money,available,inside height,size,new oven,wife happy,quick,hope
Word 7,off,worked great,warranty,powerful easy,start,sharp,large oven,went,value,reheat function,...,kitchen counter,getting,different,manual,smaller,black,15 year,arrived,quick minute,year half
Word 8,lower power,year great,worked,pleased,problem,faster,oven use,free,year excellent,reheat great,...,bit,time used,shop,perfectly,display,fit space,enjoying,happy far,minute button,year going
Word 9,cycle,great feature,review,cook easy,hard close,faster old,oven cook,ordered,quality excellent,use sensor,...,luxury,getting used,stuck,tried,problem,opening,wave,looking,release,disappointed


In [ ]:
amazon
com
old
wife happy


In [152]:
show_topics(vectorizer=vectorizer, lda_model=nmf, n_words=10)

[array(['button', 'push', 'press', 'push button', 'minute', 'start',
        'quick', 'quick minute', 'release', 'minute button'], dtype='<U22'),
 array(['great', 'great great', 'great price', 'great oven',
        'feature great', 'worked great', 'packing', 'working great',
        'great feature', 'year great'], dtype='<U22'),
 array(['door', 'close', 'slam', 'shut', 'hard', 'slam door', 'start',
        'problem', 'hard close', "won't"], dtype='<U22'),
 array(['year', '10', '10 year', '3', 'died', 'ago', '3 year', 'warranty',
        'year ago', 'second'], dtype='<U22'),
 array(['easy use', 'use', 'easy', 'use great', "it's easy",
        'powerful easy', 'pleased', 'suggest', 'clean', 'use cook'],
       dtype='<U22'),
 array(['old', 'year old', 'replaced', 'new', 'replaced old', 'old sharp',
        'sharp', 'old new', 'replace', 'replace old'], dtype='<U22'),
 array(['oven', 'great oven', 'old oven', 'new oven', 'owned',
        'recommend oven', 'oven worked', 'large oven', 'ove

In [51]:
df['lemmatized'] = df.apply(lambda x: lemmatize_text(x['token']), axis=1)

In [ ]:

# Use Gensim's NMF to get the best num of topics via coherence score
texts = df['lemmatized']

# Create a dictionary
# In gensim a dictionary is a mapping between words and their integer id
dictionary = Dictionary(texts)

# Filter out extremes to limit the number of features
dictionary.filter_extremes(
    no_below=3,
    no_above=0.85,
    keep_n=5000
)

# Create the bag-of-words format (list of (token_id, token_count))
corpus = [dictionary.doc2bow(text) for text in texts]

# Create a list of the topic numbers we want to try
topic_nums = list(np.arange(5, 75 + 1, 5))

# Run the nmf model and calculate the coherence score
# for each number of topics
coherence_scores = []

for num in topic_nums:
    nmf = Nmf(
        corpus=corpus,
        num_topics=num,
        id2word=dictionary,
        chunksize=2000,
        passes=5,
        kappa=.1,
        minimum_probability=0.01,
        w_max_iter=300,
        w_stop_condition=0.0001,
        h_max_iter=100,
        h_stop_condition=0.001,
        eval_every=10,
        normalize=True,
        random_state=42
    )
    
    # Run the coherence model to get the score
    cm = CoherenceModel(
        model=nmf,
        texts=texts,
        dictionary=dictionary,
        coherence='c_v'
    )
    
    coherence_scores.append(round(cm.get_coherence(), 5))

# Get the number of topics with the highest coherence score
scores = list(zip(topic_nums, coherence_scores))
best_num_topics = sorted(scores, key=itemgetter(1), reverse=True)[0][0]

print(best_num_topics)